In [1]:
import sys
sys.path.insert(0, "..")
from ClassifyDashboard.mClassifier import ClassifyDashboard

In [1]:
def open_ifcb(file, fnames):
    """find ifcb file, open and get volume analyzed given a file name"""
    file = os.path.splitext(file)[0]
    ix = [i for i, fname in enumerate(fnames) if file in fname][0]
    ifcb_data = ifcb.open_raw(fnames[ix])
    return IFCB_volume_analyzed(ifcb_data)


def IFCB_volume_analyzed(ifcb):
    """ From ifcb-analysis"""
    looktime = ifcb.hdr_attributes['runTime'] - ifcb.hdr_attributes['inhibitTime'] # in Seconds
    flowrate = 0.25; # .25 mls per minute
    if looktime == 0:
        """A coupl of .hdr files don't have run time values"""
        times = ifcb.adc[[22,23]].iloc[-1].values
        looktime = times[0] - times[1]
        print("{}:  looktime: {}".format(ifcb.lid,looktime))
        
        if looktime == 0:
            """Bug where last line of .adc is bad, look one earlier"""
            times = ifcb.adc[[22,23]].iloc[-2].values
            looktime = times[0] - times[1]
            print("AGAIN: {}:  looktime: {}".format(ifcb.lid,looktime))
            
    return flowrate*looktime/60

In [3]:
ClassifyDashboard().run(last_file="latest",save_output=True)

60


60it [07:53,  7.89s/it]


## Calculate Sample Volume ## 

In [8]:
import pandas as pd
import numpy as np
import glob, ifcb, os

In [9]:
last_file = sorted(glob.glob("../data/classified-csv/classified_*"))[-1]
df = pd.read_csv(last_file)
df.index = pd.to_datetime(df['dateTime'])

fnames = glob.glob("/opt/ifcb-data/power-buoy-deployment/CA-IFCB-161/2021/*/*.roi")
volumes = [open_ifcb(file, fnames) for file in df['fileName'].values]
df['volume_analyzed'] = volumes
critter_cols = [col for col in df.columns.values if col not in ["dateTime","fileName","volume_analyzed","inhibitTime",'runTime','syringeSize']]
df[critter_cols] = df[critter_cols].divide(df['volume_analyzed'],axis=0)
df.to_csv(os.path.join("../data/classified-volume/",os.path.basename(os.path.splitext(last_file)[0] + "_volume.csv")), index=False)

In [11]:
df

,Akashiwo,Alexandrium_singlet,Amy_Gony_Protoc,Asterionellopsis,Bacteriastrum,Centric,Ceratium,Chaetoceros,Ciliates,Coccolithophore,...,Tropidoneis,Umbilicosphaera,cyst,zooplankton_misc,dateTime,inhibitTime,runTime,syringeSize,fileName,volume_analyzed
dateTime,,,,,,,,,,,,,,,,,,,,,
2021-08-20 20:02:27,0.000000,0.0,0.000000,0.000000,0.000000,17.777226,4.182877,8.365753,7.320034,1.045719,...,0.0,0.000000,0.0,1.045719,2021-08-20 20:02:27,9.794590,239.301719,5.0,D20210820T200227_IFCB161.roi,0.956280
2021-08-20 20:09:49,0.000000,0.0,0.000000,2.105424,0.000000,34.739504,4.210849,3.158137,12.632547,6.316273,...,0.0,0.000000,0.0,0.000000,2021-08-20 20:09:49,12.552658,240.535191,5.0,D20210820T200949_IFCB161.roi,0.949927
2021-08-20 20:17:12,0.000000,0.0,1.046221,0.000000,0.000000,21.970648,3.138664,4.184885,7.323549,3.138664,...,0.0,0.000000,0.0,2.092443,2021-08-20 20:17:12,10.509549,239.906510,5.0,D20210820T201712_IFCB161.roi,0.955821
2021-08-20 20:38:27,0.636903,0.0,0.000000,0.212301,0.000000,23.990011,2.335311,6.581330,18.894787,5.307525,...,0.0,0.000000,0.0,1.698408,2021-08-20 20:38:27,68.691984,1199.162500,5.0,D20210820T203827_IFCB161.roi,4.710294
2021-08-20 21:25:51,0.640835,0.0,0.427224,0.213612,0.000000,24.992579,4.913071,9.612531,17.729779,4.485848,...,0.0,0.213612,0.0,1.922506,2021-08-20 21:25:51,77.167760,1200.701250,5.0,D20210820T212551_IFCB161.roi,4.681390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-10 15:43:05,0.410888,0.0,0.205444,0.410888,0.000000,23.215170,2.670772,3.287104,3.903436,1.438108,...,0.0,0.205444,0.0,0.205444,2021-09-10 15:43:05,28.432398,1196.634028,5.0,D20210910T154305_IFCB161.roi,4.867507
2021-09-10 16:06:47,0.206237,0.0,0.412475,0.206237,0.000000,23.304820,1.856136,2.474848,4.743459,0.824949,...,0.0,0.000000,0.0,0.618712,2021-09-10 16:06:47,25.250273,1188.958056,5.0,D20210910T160647_IFCB161.roi,4.848782
2021-09-10 16:30:29,0.818451,0.0,0.204613,0.204613,0.204613,22.916637,3.273805,3.683031,4.296869,1.023064,...,0.0,0.000000,0.0,0.000000,2021-09-10 16:30:29,25.659816,1198.606806,5.0,D20210910T163029_IFCB161.roi,4.887279
